In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import os

In [2]:
client = pd.read_csv('CLIENT_191102.tsv',sep='	')

In [3]:
client.loc[client['EE Provider ID'] == 'Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838)','EE Provider ID'] = 'SESH'
client.loc[client['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Entry- ESG(1970)','EE Provider ID'] = 'JENT'
client.loc[client['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Forward- ESG(5694)','EE Provider ID'] = 'JFOR'
client.loc[client['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Outreach- ESG(4515)','EE Provider ID'] = 'JOUT'
client.loc[client['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Recovery- ESG(1932)','EE Provider ID'] = 'JREC'
client.loc[client['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Tech- ESG(4516)','EE Provider ID'] = 'JTEC'
client.loc[client['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Veterans- ESG(5069)','EE Provider ID'] = 'JVET'


In [4]:
client['Duration']=client['Client Age at Exit']-client['Client Age at Entry']

In [5]:
#For EE Provider ID, it appears the vast majority of clients are part of Singles Emergency Shelter. Let's get a count of clients by EE Provider ID.
client['EE Provider ID'].value_counts()

SESH    4319
JENT     721
JREC      67
JFOR      65
JTEC      61
JOUT      51
JVET      15
Name: EE Provider ID, dtype: int64

In [6]:
#80% of clients are listed under Singles Emergency Shelter, and all other providers closed in 2015
#We will limit our analysis to Singles Emergency Shelter
client_SESH=client[client['EE Provider ID']=='SESH']

In [7]:
client_SESH['Client ID'].value_counts()

320781     29
344697     29
419124     18
124817     17
2054       16
           ..
453207      1
399969      1
1000042     1
420459      1
421940      1
Name: Client ID, Length: 2102, dtype: int64

In [8]:
client_SESH['Client Unique ID'].value_counts()

smrm06231974s500m610    29
pglf11111968p430g456    29
mlwm08071974m632l200    18
ljfm07151959l563j162    17
jdam06181979j500d200    16
                        ..
cwlf02201963c530w425     1
mewm06061978m240e363     1
twmm09231960t655w520     1
rmrm01171973r534m600     1
rdkm08311976r263d220     1
Name: Client Unique ID, Length: 2100, dtype: int64

In [9]:
client_SESH['EE UID'].value_counts()

966362     1
688821     1
867014     1
797380     1
873155     1
          ..
1045848    1
838999     1
714070     1
1057612    1
780849     1
Name: EE UID, Length: 4319, dtype: int64

In [9]:
client_SESH['Client Veteran Status'].value_counts()
#large majority are no, so can remove

No (HUD)                    3848
Yes (HUD)                    461
Data not collected (HUD)       2
Name: Client Veteran Status, dtype: int64

In [10]:
client_SESH['Duration'].value_counts()
#large majority are 0.0, so can remove

0.0    3860
1.0     455
2.0       4
Name: Duration, dtype: int64

In [11]:
#Client ID and Client Unique ID appear to have same counts, so we will remove Client Unique ID and only use Client ID in our analysis.
#EE UID appear to be totally unique for each entry, so not very useful for our analysis. We will remove that column.
#We will also remove Client Age at Exit and instead leave Duration to give same information.
client_SESH=client[client['EE Provider ID']=='SESH']
client_SESH = client_SESH.drop(["Duration","Client Veteran Status","Client Ethnicity","EE Provider ID", "EE UID", "Client Unique ID", "Client Age at Exit"], axis=1)
client_SESH

,Client ID,Client Age at Entry,Client Gender,Client Primary Race
0,397941,60.0,Female,White (HUD)
1,130335,48.0,Female,Black or African American (HUD)
2,188933,42.0,Female,Black or African American (HUD)
3,168290,57.0,Male,White (HUD)
4,123122,51.0,Male,White (HUD)
...,...,...,...,...
4314,166411,31.0,Male,White (HUD)
4315,198376,39.0,Male,White (HUD)
4316,1004266,59.0,Male,Black or African American (HUD)
4317,407506,22.0,Male,Black or African American (HUD)


In [12]:
dis_ent = pd.read_csv('DISABILITY_ENTRY_191102.tsv.txt', sep = '	')

In [13]:
dis_ent.loc[dis_ent['EE Provider ID'] == 'Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838)','EE Provider ID'] = 'SESH'
dis_ent.loc[dis_ent['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Entry- ESG(1970)','EE Provider ID'] = 'JENT'
dis_ent.loc[dis_ent['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Forward- ESG(5694)','EE Provider ID'] = 'JFOR'
dis_ent.loc[dis_ent['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Outreach- ESG(4515)','EE Provider ID'] = 'JOUT'
dis_ent.loc[dis_ent['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Recovery- ESG(1932)','EE Provider ID'] = 'JREC'
dis_ent.loc[dis_ent['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Tech- ESG(4516)','EE Provider ID'] = 'JTEC'
dis_ent.loc[dis_ent['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Veterans- ESG(5069)','EE Provider ID'] = 'JVET'

In [14]:
dis_ex = pd.read_csv('DISABILITY_EXIT_191102.tsv.txt', sep = '	')

In [15]:
dis_ex.loc[dis_ex['EE Provider ID'] == 'Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838)','EE Provider ID'] = 'SESH'
dis_ex.loc[dis_ex['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Entry- ESG(1970)','EE Provider ID'] = 'JENT'
dis_ex.loc[dis_ex['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Forward- ESG(5694)','EE Provider ID'] = 'JFOR'
dis_ex.loc[dis_ex['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Outreach- ESG(4515)','EE Provider ID'] = 'JOUT'
dis_ex.loc[dis_ex['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Recovery- ESG(1932)','EE Provider ID'] = 'JREC'
dis_ex.loc[dis_ex['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Tech- ESG(4516)','EE Provider ID'] = 'JTEC'
dis_ex.loc[dis_ex['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Veterans- ESG(5069)','EE Provider ID'] = 'JVET'

In [16]:
dis_ent_discount=dis_ent['Disability Type (Entry)'].value_counts()
dis_ent_discount

Mental Health Problem (HUD)          5324
Physical (HUD)                       5128
Chronic Health Condition (HUD)       5110
Both Alcohol and Drug Abuse (HUD)    5085
Drug Abuse (HUD)                     5076
HIV/AIDS (HUD)                       5061
Developmental (HUD)                  5048
Alcohol Abuse (HUD)                  5035
Physical/Medical                      501
Dual Diagnosis                          6
Other                                   6
Vision Impaired                         3
Other: Learning                         3
Hearing Impaired                        2
Other: Speech                           2
Name: Disability Type (Entry), dtype: int64

In [18]:
dis_ex['Disability Type (Exit)'].value_counts()
#Entry and exit disability counts are very similar

Mental Health Problem (HUD)          5389
Physical (HUD)                       5200
Chronic Health Condition (HUD)       5178
Both Alcohol and Drug Abuse (HUD)    5153
Drug Abuse (HUD)                     5151
HIV/AIDS (HUD)                       5128
Developmental (HUD)                  5117
Alcohol Abuse (HUD)                  5105
Physical/Medical                      501
Dual Diagnosis                          6
Other                                   6
Vision Impaired                         3
Other: Learning                         3
Hearing Impaired                        2
Other: Speech                           2
Name: Disability Type (Exit), dtype: int64

In [19]:
dis_ent['Date Added (417-date_added)'].value_counts()
#Date added looks heavily skewed to a few dates, likely due to moving over data from other source. Not very useful for this analysis.

7/18/2015     2543
7/17/2015     1384
12/28/2015    1374
12/22/2015    1055
7/16/2015      787
              ... 
7/16/2014        1
10/10/2017       1
2/23/2015        1
3/6/2009         1
1/23/2015        1
Name: Date Added (417-date_added), Length: 959, dtype: int64

In [20]:
dis_ent['Disability Determination (Entry)'].value_counts()

No (HUD)                     36275
Yes (HUD)                     4934
Client doesn't know (HUD)       37
Data not collected (HUD)        11
Name: Disability Determination (Entry), dtype: int64

In [44]:
#Appears to be little difference in disabilities between Entry and Exit (likely due to short time period)
#We will limit analysis to disability status at entry
del(dis_ex)
dis_ent_SESH = dis_ent[dis_ent['EE Provider ID']=='SESH']
dis_ent_SESH=dis_ent_SESH[['Client ID', 'Disability Determination (Entry)', 'Disability Type (Entry)']]

In [21]:
ee_ud = pd.read_csv('EE_UDES_191102.tsv.txt', sep = '	')

In [22]:
ee_ud.loc[ee_ud['EE Provider ID'] == 'Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838)','EE Provider ID'] = 'SESH'
ee_ud.loc[ee_ud['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Entry- ESG(1970)','EE Provider ID'] = 'JENT'
ee_ud.loc[ee_ud['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Forward- ESG(5694)','EE Provider ID'] = 'JFOR'
ee_ud.loc[ee_ud['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Outreach- ESG(4515)','EE Provider ID'] = 'JOUT'
ee_ud.loc[ee_ud['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Recovery- ESG(1932)','EE Provider ID'] = 'JREC'
ee_ud.loc[ee_ud['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Tech- ESG(4516)','EE Provider ID'] = 'JTEC'
ee_ud.loc[ee_ud['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Veterans- ESG(5069)','EE Provider ID'] = 'JVET'

In [23]:
ee_ud['Client Location(4378)'].value_counts()
#almost all of Client Locatin Values are Durham City and County CoC

NC-502 Durham City and County CoC          5201
NC-505 Charlotte/Mecklenburg County CoC       5
NC-503 NC Balance of State CoC                1
NC-507 Raleigh/Wake County CoC                1
Name: Client Location(4378), dtype: int64

In [24]:
ee_ud['Relationship to Head of Household(4374)'].value_counts()
#almost all of Relationship to Head of Household are Self

Self (head of household)                                                           5185
Head of household's spouse or partner                                                 2
Head of household's child                                                             2
Head of household's other relation member (other relation to head of household)       1
Name: Relationship to Head of Household(4374), dtype: int64

In [25]:
ee_ud['Prior Living Situation(43)'].value_counts()
#This variable is interesting, with large number of entries for many different values. We will keep it for analysis.

Place not meant for habitation (HUD)                                                                  1469
Emergency shelter, incl. hotel/motel paid for w/ ES voucher, or RHY-funded Host Home shelter (HUD)     891
Staying or living in a family member's room, apartment or house (HUD)                                  804
Staying or living in a friend's room, apartment or house (HUD)                                         730
Rental by client, no ongoing housing subsidy (HUD)                                                     271
Hotel or motel paid for without emergency shelter voucher (HUD)                                        225
Jail, prison or juvenile detention facility (HUD)                                                      216
Hospital or other residential non-psychiatric medical facility (HUD)                                   147
Transitional housing for homeless persons (including homeless youth) (HUD)                             124
Substance abuse treatment facility or

In [26]:
ee_ud['Length of Stay in Previous Place(1934)'].value_counts()
#This variable is also interesting, with large number of entries for many different values. We will keep it for analysis.

One year or longer (HUD)                     1168
90 days or more, but less than one year      1117
One month or more, but less than 90 days     1019
One week or more, but less than one month     885
Two to six nights                             611
One night or less                             337
One week or less (HUD)                         73
Data not collected (HUD)                       12
Client refused (HUD)                            2
Client doesn't know (HUD)                       2
Name: Length of Stay in Previous Place(1934), dtype: int64

In [27]:
ee_ud['On the night before did you stay on the streets, ES or SH?(5165)'].value_counts()
#This variable has NA value for nearly all, so not worth including.

Yes    10
No      4
Name: On the night before did you stay on the streets, ES or SH?(5165), dtype: int64

In [28]:
ee_ud['Regardless of where they stayed last night - Number of times the client has been on the streets, in ES, or SH in the past three years including today(5167)'].value_counts()
#This variable has some interesting information, so worth including. Probably should change the label though.

Four or more times (HUD)     1795
One time (HUD)               1730
Two times (HUD)               920
Three times (HUD)             541
Data not collected (HUD)       15
Client doesn't know (HUD)       8
Client refused (HUD)            4
Name: Regardless of where they stayed last night - Number of times the client has been on the streets, in ES, or SH in the past three years including today(5167), dtype: int64

In [29]:
ee_ud['Total number of months homeless on the street, in ES or SH in the past three years(5168)'].value_counts()
#This variable also has interesting info and is worth keeping and changing label.

More than 12 months (HUD)                         1898
One month (this time is the first month) (HUD)    1168
Data not collected (HUD)                           742
2                                                  325
3                                                  198
6                                                  153
4                                                  124
5                                                   84
8                                                   75
12                                                  69
9                                                   47
7                                                   43
10                                                  39
Client doesn't know (HUD)                           22
11                                                  21
Client refused (HUD)                                 4
Name: Total number of months homeless on the street, in ES or SH in the past three years(5168), dtype: int64

In [30]:
ee_ud['Housing Status(2703)'].value_counts()
#This variable is almost entirely Category 1, so not best to keep for analysis.

Category 1 - Homeless (HUD)                                      4608
Stably housed (HUD)                                                60
Category 2 - At imminent risk of losing housing (HUD)              42
At-risk of homelessness (HUD)                                      16
Client doesn't know (HUD)                                           4
Client refused (HUD)                                                2
Category 3 - Homeless only under other federal statutes (HUD)       1
Name: Housing Status(2703), dtype: int64

In [31]:
ee_ud['Does the client have a disabling condition?(1935)'].value_counts()
#This variable is worth keeping for now, with close to even numbers for Yes and No

Yes (HUD)                    3000
No (HUD)                     2219
Client doesn't know (HUD)       3
Client refused (HUD)            2
Data not collected (HUD)        1
Name: Does the client have a disabling condition?(1935), dtype: int64

In [32]:
ee_ud['Covered by Health Insurance(4376)'].value_counts()
#This variable is also worth keeping for now, with large numbers for both Yes and No

No (HUD)                     3135
Yes (HUD)                    1969
Client doesn't know (HUD)      11
Data not collected (HUD)        5
Client refused (HUD)            2
Name: Covered by Health Insurance(4376), dtype: int64

In [33]:
ee_ud['Domestic violence victim/survivor(341)'].value_counts()
#Over 10% answered yes, significant enough to keep this variable for now.

No (HUD)                     4710
Yes (HUD)                     497
Client doesn't know (HUD)      10
Client refused (HUD)            1
Name: Domestic violence victim/survivor(341), dtype: int64

In [34]:
ee_ud['If yes for Domestic violence victim/survivor, when experience occurred(1917)'].value_counts()
#Considering this subset is already small, and the largest portion are from more than a year ago, we can take out this variable

More than a year ago (HUD)             225
Within the past three months (HUD)     134
Three to six months ago (HUD)           48
From six to twelve months ago (HUD)     38
Data not collected (HUD)                27
Client refused (HUD)                     7
Client doesn't know (HUD)                4
Name: If yes for Domestic violence victim/survivor, when experience occurred(1917), dtype: int64

In [35]:
ee_ud['Entry Exit Provider Program Type Code'].value_counts()
#Only one value for this variable, so nothing to analyze

Emergency Shelter (HUD)    5299
Name: Entry Exit Provider Program Type Code, dtype: int64

In [36]:
ee_ud['Zip Code (of Last Permanent Address, if known)(1932)'].value_counts()
#This can be removed, not likely to be useful for analysis

27701    1579
27703     655
27707     339
27704     245
27705     212
         ... 
27937       1
na          1
23224       1
08360       1
34953       1
Name: Zip Code (of Last Permanent Address, if known)(1932), Length: 440, dtype: int64

In [38]:
ee_ud_SESH = ee_ud[ee_ud['EE Provider ID']=='SESH']
ee_ud_SESH=ee_ud_SESH.drop(['If yes for Domestic violence victim/survivor, when experience occurred(1917)','Date of Birth(893)','Zip Code (of Last Permanent Address, if known)(1932)','EE Provider ID','Entry Exit Provider Program Type Code','EE UID','Client Unique ID','Client Location(4378)','Relationship to Head of Household(4374)','On the night before did you stay on the streets, ES or SH?(5165)','Housing Status(2703)','Did you stay less than 7 nights?(5164)','Did you stay less than 90 days?(5163)'], axis=1)
del(ee_ud)

In [39]:
noncash_ent = pd.read_csv('NONCASH_ENTRY_191102.tsv.txt', sep = '	')

In [40]:
noncash_ent.loc[noncash_ent['EE Provider ID'] == 'Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838)','EE Provider ID'] = 'SESH'
noncash_ent.loc[noncash_ent['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Entry- ESG(1970)','EE Provider ID'] = 'JENT'
noncash_ent.loc[noncash_ent['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Forward- ESG(5694)','EE Provider ID'] = 'JFOR'
noncash_ent.loc[noncash_ent['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Outreach- ESG(4515)','EE Provider ID'] = 'JOUT'
noncash_ent.loc[noncash_ent['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Recovery- ESG(1932)','EE Provider ID'] = 'JREC'
noncash_ent.loc[noncash_ent['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Tech- ESG(4516)','EE Provider ID'] = 'JTEC'
noncash_ent.loc[noncash_ent['EE Provider ID'] == 'XXXClosed2015 Urban Ministries of Durham- Durham County- Journey Veterans- ESG(5069)','EE Provider ID'] = 'JVET'

In [41]:
noncash_ent['Non-Cash Source (Entry)'].value_counts()
#Worth keeping

Supplemental Nutrition Assistance Program (Food Stamps) (HUD)          5279
TANF Child Care Services (HUD)                                         4949
Special Supplemental Nutrition Program for WIC (HUD)                   4948
Other TANF-Funded Services (HUD)                                       4948
TANF Transportation Services (HUD)                                     4948
Other Source (HUD)                                                     4947
Section 8, Public Housing, or other ongoing rental assistance (HUD)    3931
Temporary rental assistance (HUD)                                      3931
Name: Non-Cash Source (Entry), dtype: int64

In [42]:
noncash_ent_SESH = noncash_ent[noncash_ent['EE Provider ID']=='SESH']
noncash_ent_SESH=noncash_ent_SESH[['Client ID', 'Non-Cash Source (Entry)']]
del(noncash_ent)

In [50]:
umd = client_SESH.merge(ee_ud_SESH, on='Client ID', how='left')
umd

,Client ID,Client Age at Entry,Client Gender,Client Primary Race,Prior Living Situation(43),Length of Stay in Previous Place(1934),"Regardless of where they stayed last night - Number of times the client has been on the streets, in ES, or SH in the past three years including today(5167)","Total number of months homeless on the street, in ES or SH in the past three years(5168)",Does the client have a disabling condition?(1935),Covered by Health Insurance(4376),Domestic violence victim/survivor(341)
0,397941,60.0,Female,White (HUD),"Staying or living in a friend's room, apartmen...",One year or longer (HUD),Two times (HUD),2,Yes (HUD),Yes (HUD),No (HUD)
1,130335,48.0,Female,Black or African American (HUD),"Staying or living in a family member's room, a...","One month or more, but less than 90 days",Four or more times (HUD),More than 12 months (HUD),Yes (HUD),Yes (HUD),Yes (HUD)
2,130335,48.0,Female,Black or African American (HUD),"Staying or living in a family member's room, a...","One month or more, but less than 90 days",Four or more times (HUD),More than 12 months (HUD),Yes (HUD),Yes (HUD),Yes (HUD)
3,130335,48.0,Female,Black or African American (HUD),"Staying or living in a family member's room, a...","One month or more, but less than 90 days",Four or more times (HUD),More than 12 months (HUD),Yes (HUD),Yes (HUD),Yes (HUD)
4,130335,48.0,Female,Black or African American (HUD),"Staying or living in a family member's room, a...","One month or more, but less than 90 days",Four or more times (HUD),More than 12 months (HUD),Yes (HUD),Yes (HUD),Yes (HUD)
...,...,...,...,...,...,...,...,...,...,...,...
18612,429149,52.0,Female,Black or African American (HUD),"Staying or living in a family member's room, a...",One year or longer (HUD),One time (HUD),One month (this time is the first month) (HUD),Yes (HUD),Yes (HUD),Yes (HUD)
18613,429149,52.0,Female,Black or African American (HUD),Place not meant for habitation (HUD),One year or longer (HUD),Two times (HUD),More than 12 months (HUD),Yes (HUD),Yes (HUD),Yes (HUD)
18614,429149,52.0,Female,Black or African American (HUD),"Rental by client, with other ongoing housing s...","90 days or more, but less than one year",Three times (HUD),More than 12 months (HUD),Yes (HUD),Yes (HUD),Yes (HUD)
18615,429149,52.0,Female,Black or African American (HUD),"Staying or living in a friend's room, apartmen...",One night or less,Three times (HUD),More than 12 months (HUD),No (HUD),Yes (HUD),Yes (HUD)


In [63]:
umd=umd.rename(columns={'Regardless of where they stayed last night - Number of times the client has been on the streets, in ES, or SH in the past three years including today(5167)':'Recent_Times_Homeless'})

In [64]:
umd=umd.rename(columns={'Client ID':'ID'})

In [65]:
umd=umd.rename(columns={'Client Age at Entry':'Entry_Age'})

In [66]:
umd=umd.rename(columns={'Client Gender':'Gender'})

In [67]:
umd=umd.rename(columns={'Client Primary Race':'Race'})

In [68]:
umd=umd.rename(columns={'Prior Living Situation(43)':'Prior_Living'})

In [69]:
umd=umd.rename(columns={'Length of Stay in Previous Place(1934)':'Prior_Living_Time'})

In [70]:
umd=umd.rename(columns={'Total number of months homeless on the street, in ES or SH in the past three years(5168)':'Recent_Months_Homeless'})

In [71]:
umd=umd.rename(columns={'Does the client have a disabling condition?(1935)':'Disability'})

In [72]:
umd=umd.rename(columns={'Covered by Health Insurance(4376)':'Health_Insurance'})

In [73]:
umd=umd.rename(columns={'Domestic violence victim/survivor(341)':'Domestic_Violence'})

In [88]:
print(umd['Race'].unique())

['White' 'Black/AA' 'American Indian or Alaska Native (HUD)' 'Asian (HUD)'
 'Native Hawaiian or Other Pacific Islander (HUD)'
 "Client doesn't know (HUD)" 'Data not collected (HUD)'
 'Client refused (HUD)']


In [80]:
umd.loc[umd['Race'] == 'White (HUD)','Race'] = 'White'

In [86]:
umd.loc[umd['Race'] == 'Black or African American (HUD)','Race'] = 'Black/AA'

In [190]:
umd.loc[umd['Race'] == '1','Race'] = 'Black/AA'

In [192]:
umd.loc[umd['Race'] == 'American Indian or Alaska Native','Race'] = 'Am Indian/Alaskan'

In [91]:
umd.loc[umd['Race'] == 'Asian (HUD)','Race'] = 'Asian'

In [92]:
umd.loc[umd['Race'] == 'Native Hawaiian or Other Pacific Islander (HUD)','Race'] = 'Hawaiian/Pacific Islander'

In [93]:
umd.loc[umd['Race'] == "Client doesn't know (HUD)",'Race'] = 'Client Unsure'

In [94]:
umd.loc[umd['Race'] == 'Data not collected (HUD)','Race'] = 'No Data Collected'

In [95]:
umd.loc[umd['Race'] == 'Client refused (HUD)','Race'] = 'Client Refused'

In [100]:
print(umd['Recent_Months_Homeless'].unique())

['2' 'More than 12 months (HUD)' 'Data not collected (HUD)' '6' nan '10'
 '7' '3' '4' 'One month (this time is the first month) (HUD)' '11'
 "Client doesn't know (HUD)" '9' '8' '12' '5' 'Client refused (HUD)']


In [102]:
umd.loc[umd['Recent_Months_Homeless'] == 'One month (this time is the first month) (HUD)','Recent_Months_Homeless'] = '1'

In [104]:
umd.loc[umd['Recent_Months_Homeless'] == 'More than 12 months (HUD)','Recent_Months_Homeless'] = '12+'

In [106]:
umd.loc[umd['Recent_Months_Homeless'] == 'Data not collected (HUD)','Recent_Months_Homeless'] = 'NA'

In [107]:
umd.loc[umd['Recent_Months_Homeless'] == "Client doesn't know (HUD)",'Recent_Months_Homeless'] = 'DK'

In [108]:
umd.loc[umd['Recent_Months_Homeless'] == 'Client refused (HUD)','Recent_Months_Homeless'] = 'CR'

In [110]:
print(umd['Recent_Times_Homeless'].unique())

['Two times (HUD)' 'Four or more times (HUD)' 'Three times (HUD)'
 'One time (HUD)' nan 'Data not collected (HUD)'
 "Client doesn't know (HUD)" 'Client refused (HUD)']


In [113]:
umd.loc[umd['Recent_Times_Homeless'] == 'Two times (HUD)','Recent_Times_Homeless'] = 2

In [115]:
umd.loc[umd['Recent_Times_Homeless'] == 'Three times (HUD)','Recent_Times_Homeless'] = 3

In [118]:
umd.loc[umd['Recent_Times_Homeless'] == 'Four or more times (HUD)','Recent_Times_Homeless'] = '4+'

In [120]:
umd.loc[umd['Recent_Times_Homeless'] == 'One time (HUD)','Recent_Times_Homeless'] = 1

In [122]:
umd.loc[umd['Recent_Times_Homeless'] == 'Data not collected (HUD)','Recent_Times_Homeless'] = 'NA'

In [124]:
umd.loc[umd['Recent_Times_Homeless'] == "Client doesn't know (HUD)",'Recent_Times_Homeless'] = 'DK'

In [125]:
umd.loc[umd['Recent_Times_Homeless'] == 'Client refused (HUD)','Recent_Times_Homeless'] = 'CR'

In [127]:
print(umd['Prior_Living'].unique())

["Staying or living in a friend's room, apartment or house (HUD)"
 "Staying or living in a family member's room, apartment or house (HUD)"
 'Place not meant for habitation (HUD)'
 'Emergency shelter, incl. hotel/motel paid for w/ ES voucher, or RHY-funded Host Home shelter (HUD)'
 'Rental by client, with VASH housing subsidy (HUD)'
 'Rental by client, no ongoing housing subsidy (HUD)' nan
 'Rental by client, with other ongoing housing subsidy (HUD)'
 'Jail, prison or juvenile detention facility (HUD)'
 'Transitional housing for homeless persons (including homeless youth) (HUD)'
 'Hotel or motel paid for without emergency shelter voucher (HUD)'
 'Residential project or halfway house with no homeless criteria (HUD)'
 'Substance abuse treatment facility or detox center (HUD)'
 'Owned by client, with ongoing housing subsidy (HUD)'
 'Hospital or other residential non-psychiatric medical facility (HUD)'
 'Permanent housing (other than RRH) for formerly homeless persons (HUD)'
 "Client doesn'

In [128]:
umd.loc[umd['Prior_Living'] == "Staying or living in a friend's room, apartment or house (HUD)",'Prior_Living'] = "Friend Home"

In [195]:
umd.loc[umd['Prior_Living'] == "Data not collected (HUD)",'Prior_Living'] = "Data not collected"

In [130]:
umd.loc[umd['Prior_Living'] == "Staying or living in a family member's room, apartment or house (HUD)",'Prior_Living'] = "Family Home"

In [132]:
umd.loc[umd['Prior_Living'] == 'Place not meant for habitation (HUD)','Prior_Living'] = 'Place not meant for habitation'

In [133]:
umd.loc[umd['Prior_Living'] == "Emergency shelter, incl. hotel/motel paid for w/ ES voucher, or RHY-funded Host Home shelter (HUD)",'Prior_Living'] = "Emergency shelter"

In [134]:
umd.loc[umd['Prior_Living'] == "Rental by client, with VASH housing subsidy (HUD)",'Prior_Living'] = "VASH subsidy rental"

In [135]:
umd.loc[umd['Prior_Living'] == "Rental by client, no ongoing housing subsidy (HUD)",'Prior_Living'] = "Rental w/o subsidy"

In [136]:
umd.loc[umd['Prior_Living'] == "Rental by client, with other ongoing housing subsidy (HUD)",'Prior_Living'] = "Rental w/ other subsidy"

In [137]:
umd.loc[umd['Prior_Living'] == "Jail, prison or juvenile detention facility (HUD)",'Prior_Living'] = "Incarcerated"

In [138]:
umd.loc[umd['Prior_Living'] == "Transitional housing for homeless persons (including homeless youth) (HUD)",'Prior_Living'] = "Transitional housing"

In [139]:
umd.loc[umd['Prior_Living'] == "Hotel or motel paid for without emergency shelter voucher (HUD)",'Prior_Living'] = "Hotel or motel"

In [140]:
umd.loc[umd['Prior_Living'] == "Residential project or halfway house with no homeless criteria (HUD)",'Prior_Living'] = "Halfway house"

In [194]:
umd.loc[umd['Prior_Living'] == "Substance abuse treatment facility or detox center (HUD)",'Prior_Living'] = "Substance treatment center"

In [141]:
umd.loc[umd['Prior_Living'] == "Owned by client, with ongoing housing subsidy (HUD)",'Prior_Living'] = "Owned w/ subsidy"

In [142]:
umd.loc[umd['Prior_Living'] == "Hospital or other residential non-psychiatric medical facility (HUD)",'Prior_Living'] = "Medical facility"

In [143]:
umd.loc[umd['Prior_Living'] == "Permanent housing (other than RRH) for formerly homeless persons (HUD)",'Prior_Living'] = "Permanent housing for ex-homeless"

In [144]:
umd.loc[umd['Prior_Living'] == "Client doesn't know (HUD)",'Prior_Living'] = "Client does not know"

In [145]:
umd.loc[umd['Prior_Living'] == "Owned by client, no ongoing housing subsidy (HUD)",'Prior_Living'] = "Owned w/o subsidy"

In [146]:
umd.loc[umd['Prior_Living'] == "Other (HUD)",'Prior_Living'] = "Other"

In [147]:
umd.loc[umd['Prior_Living'] == "Safe Haven (HUD)",'Prior_Living'] = "Safe haven"

In [148]:
umd.loc[umd['Prior_Living'] == "Psychiatric hospital or other psychiatric facility (HUD)",'Prior_Living'] = "Psychiatric facility"

In [149]:
umd.loc[umd['Prior_Living'] == "Long-term care facility or nursing home (HUD)",'Prior_Living'] = "Long-term care facility"

In [150]:
umd.loc[umd['Prior_Living'] == "Foster care home or foster care group home (HUD)",'Prior_Living'] = "Foster care home"

In [151]:
umd.loc[umd['Prior_Living'] == "Interim Housing (HUD) (Retired)",'Prior_Living'] = "Interim housing (Retired)"

In [152]:
umd.loc[umd['Prior_Living'] == "Rental by client, with GPD TIP housing subsidy (HUD)",'Prior_Living'] = "Rental w/ GPD TIP subsidy"

In [153]:
umd.loc[umd['Prior_Living'] == "Client refused (HUD)",'Prior_Living'] = "Client refused"

In [154]:
umd.loc[umd['Prior_Living'] == "Rental by client in a public housing unit (HUD)",'Prior_Living'] = "Public housing rental"

In [156]:
print(umd['Disability'].unique())

['Yes (HUD)' 'No (HUD)' nan 'Data not collected (HUD)'
 'Client refused (HUD)']


In [157]:
umd.loc[umd['Disability'] == "Yes (HUD)",'Disability'] = "Yes"

In [158]:
umd.loc[umd['Disability'] == "No (HUD)",'Disability'] = "No"

In [159]:
umd.loc[umd['Disability'] == "Data not collected (HUD)",'Disability'] = "Data not collected"

In [160]:
umd.loc[umd['Disability'] == "Client refused (HUD)",'Disability'] = "Client refused"

In [162]:
print(umd['Health_Insurance'].unique())

['Yes (HUD)' 'No (HUD)' nan "Client doesn't know (HUD)"
 'Data not collected (HUD)' 'Client refused (HUD)']


In [163]:
umd.loc[umd['Health_Insurance'] == "Yes (HUD)",'Health_Insurance'] = "Yes"

In [164]:
umd.loc[umd['Health_Insurance'] == "No (HUD)",'Health_Insurance'] = "No"

In [165]:
umd.loc[umd['Health_Insurance'] == "Client doesn't know (HUD)",'Health_Insurance'] = "Client does not know"

In [166]:
umd.loc[umd['Health_Insurance'] == "Data not collected (HUD)",'Health_Insurance'] = "Data not collected"

In [167]:
umd.loc[umd['Health_Insurance'] == "Client refused (HUD)",'Health_Insurance'] = "Client refused"

In [169]:
print(umd['Domestic_Violence'].unique())

['No (HUD)' 'Yes (HUD)' nan "Client doesn't know (HUD)"
 'Client refused (HUD)']


In [170]:
umd.loc[umd['Domestic_Violence'] == "Yes (HUD)",'Domestic_Violence'] = "Yes"

In [171]:
umd.loc[umd['Domestic_Violence'] == "No (HUD)",'Domestic_Violence'] = "No"

In [172]:
umd.loc[umd['Domestic_Violence'] == "Client doesn't know (HUD)",'Domestic_Violence'] = "Client does not know"

In [173]:
umd.loc[umd['Domestic_Violence'] == "Client refused (HUD)",'Domestic_Violence'] = "Client refused"

In [175]:
print(umd['Prior_Living_Time'].unique())

['One year or longer (HUD)' 'One month or more, but less than 90 days'
 'One week or more, but less than one month'
 '90 days or more, but less than one year' 'Two to six nights' nan
 'One night or less' 'One week or less (HUD)' 'Data not collected (HUD)'
 "Client doesn't know (HUD)" 'Client refused (HUD)']


In [176]:
umd.loc[umd['Prior_Living_Time'] == "One year or longer (HUD)",'Prior_Living_Time'] = "1+ Years"

In [177]:
umd.loc[umd['Prior_Living_Time'] == "One month or more, but less than 90 days",'Prior_Living_Time'] = "30-89 Days"

In [179]:
umd.loc[umd['Prior_Living_Time'] == "One week or more, but less than one month",'Prior_Living_Time'] = "7-29 Days"

In [180]:
umd.loc[umd['Prior_Living_Time'] == "90 days or more, but less than one year",'Prior_Living_Time'] = "90-365 Days"

In [182]:
umd.loc[umd['Prior_Living_Time'] == "Two to six nights",'Prior_Living_Time'] = "2-6 Days"

In [183]:
umd.loc[umd['Prior_Living_Time'] == "One night or less",'Prior_Living_Time'] = "1 Day or Less"

In [184]:
umd.loc[umd['Prior_Living_Time'] == "One week or less (HUD)",'Prior_Living_Time'] = "1 Week or Less"

In [185]:
umd.loc[umd['Prior_Living_Time'] == "Data not collected (HUD)",'Prior_Living_Time'] = "Data not collected"

In [186]:
umd.loc[umd['Prior_Living_Time'] == "Client doesn't know (HUD)",'Prior_Living_Time'] = "Client doesn't know"

In [187]:
umd.loc[umd['Prior_Living_Time'] == "Client refused (HUD)",'Prior_Living_Time'] = "Client refused (HUD)"

In [196]:
umd

,ID,Entry_Age,Gender,Race,Prior_Living,Prior_Living_Time,Recent_Times_Homeless,Recent_Months_Homeless,Disability,Health_Insurance,Domestic_Violence
0,397941,60.0,Female,White,Friend Home,1+ Years,2,2,Yes,Yes,No
1,130335,48.0,Female,Black/AA,Family Home,30-89 Days,4+,12+,Yes,Yes,Yes
2,130335,48.0,Female,Black/AA,Family Home,30-89 Days,4+,12+,Yes,Yes,Yes
3,130335,48.0,Female,Black/AA,Family Home,30-89 Days,4+,12+,Yes,Yes,Yes
4,130335,48.0,Female,Black/AA,Family Home,30-89 Days,4+,12+,Yes,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...
18612,429149,52.0,Female,Black/AA,Family Home,1+ Years,1,1,Yes,Yes,Yes
18613,429149,52.0,Female,Black/AA,Place not meant for habitation,1+ Years,2,2,Yes,Yes,Yes
18614,429149,52.0,Female,Black/AA,Rental w/ other subsidy,90-365 Days,3,12+,Yes,Yes,Yes
18615,429149,52.0,Female,Black/AA,Friend Home,1 Day or Less,3,12+,No,Yes,Yes


In [200]:
dis_ent_SESH=dis_ent_SESH.rename(columns={'Client ID':'ID'})

In [204]:
dis_ent_SESH=dis_ent_SESH.rename(columns={'Disability Type (Entry)':'Disability_Type'})

In [208]:
dis_ent_SESH

,ID,Disability Determination (Entry),Disability_Type
0,397941,No (HUD),Alcohol Abuse (HUD)
1,397941,No (HUD),Both Alcohol and Drug Abuse (HUD)
2,397941,No (HUD),Chronic Health Condition (HUD)
3,397941,No (HUD),Developmental (HUD)
4,397941,No (HUD),Drug Abuse (HUD)
...,...,...,...
36391,429149,No (HUD),Physical (HUD)
36392,429149,Yes (HUD),Chronic Health Condition (HUD)
36393,429149,Yes (HUD),Chronic Health Condition (HUD)
36394,429149,Yes (HUD),Drug Abuse (HUD)


In [209]:
umd = umd.merge(dis_ent_SESH, on='ID', how='left')
umd

MemoryError: 

In [203]:
print(umd['Disability Type (Entry)'].unique())

['Alcohol Abuse (HUD)' 'Both Alcohol and Drug Abuse (HUD)'
 'Chronic Health Condition (HUD)' 'Developmental (HUD)' 'Drug Abuse (HUD)'
 'HIV/AIDS (HUD)' 'Physical (HUD)' 'Mental Health Problem (HUD)'
 'Physical/Medical' nan 'Other' 'Other: Learning' 'Hearing Impaired'
 'Other: Speech' 'Vision Impaired' 'Dual Diagnosis']


In [206]:
umd.loc[umd['Disability_Type'] == "Alcohol Abuse (HUD)",'Disability_Type'] = "Alcohol Abuse"

KeyError: 'Disability_Type'

In [210]:
umd.to_csv(r'C:/Users/Lenovo/OneDrive/Documents/UNC/BIOS 611/R/Scripts/proj3_umd.csv')

In [79]:
dis_ben.to_csv(r'C:/Users/Lenovo/OneDrive/Documents/UNC/BIOS 611/R/Scripts/proj3_dis_ben.csv')